In [1]:
from datetime import datetime
import elasticsearch
import glob
import re
import json
import os.path
from elasticsearch_dsl import Search
import time
import collections
from math import log

In [2]:
es = elasticsearch.Elasticsearch()

In [3]:
results = elasticsearch.helpers.scan(es,
    index="apdata",
    doc_type="doc",
    preserve_order=True,
    query={"query": {"match_all": {}}},
)
ids = []
doc_no = {}
for item in results:
    ids.append(item['_id'])
    doc_no[item['_id']] = item['_source']['DOCNO']


In [4]:
# start = time.time()
# batch = int(len(ids) / 100)
# vocab = {}
# vocab_index = 0
# reverse_vocabulary = []
# sparse_vectorspace = []

# # batch = 2
# for x in range(0,batch):
#     response = es.mtermvectors(index = 'apdata', 
#                    doc_type = 'doc', 
#                    ids = ids[(x * 100):((x + 1) * 100)], 
#                    fields = ["TEXT"], 
#                    payloads=False, 
#                    positions=False,
#                    offsets=False,
#                    term_statistics=True,
#                    field_statistics=True)
    
#     for i in range(0,100):
#         sparse_doc = {}
#         if(response["docs"][i]['term_vectors'] != {}):
#             terms = response["docs"][i]['term_vectors']['TEXT']['terms']
#             for term in terms.keys():
#                 if term not in vocab.keys():
#                     vocab[term] = vocab_index
#                     vocab_index = vocab_index + 1
#                     reverse_vocabulary.append(term)
#                 sparse_doc[vocab[term]] = terms[term]['term_freq']
#         else:
#             print(response["docs"][i]['_id'])
#         sparse_vectorspace.append(sparse_doc)

# response = es.mtermvectors(index = 'apdata', 
#                doc_type = 'doc', 
#                ids = ids[(x * 100):len(ids)], 
#                fields = ["TEXT"], 
#                payloads=False, 
#                positions=False,
#                offsets=False,
#                term_statistics=True,
#                field_statistics=True)

# for i in range(0, (len(ids) % 100)):
#     sparse_doc = {}
#     if(response["docs"][i]['term_vectors'] != {}):
#         terms = response["docs"][i]['term_vectors']['TEXT']['terms']
#         for term in terms.keys():
#             if term not in vocab.keys():
#                 vocab[term] = vocab_index
#                 vocab_index = vocab_index + 1
#                 reverse_vocabulary.append(term)
#             sparse_doc[vocab[term]] = terms[term]['term_freq']

#     else:
#         print(response["docs"][i]['_id'])
#     sparse_vectorspace.append(sparse_doc)



# # terms = response["docs"][0]['term_vectors']['TEXT']['terms']
# # for term in terms.keys():
# #     if term not in vocab.keys():
# #         vocab[term] = vocab_index
# #         vocab_index = vocab_index + 1
# #         reverse_vocabulary.append(term)

# end = time.time()
# print(end - start)


In [5]:
# count = 0
# for vector in sparse_vectorspace:
#     if 22515 in vector.keys():
#         count = count + 1

In [6]:
# start = time.time()
# for i in range(0, len(vsp)):
#     for j in range(0, len(vocab)):
#         i,j
# end = time.time()
# print(end - start)

In [7]:
x = 0
start = time.time()
vocab = {}
vocab_index = 0
reverse_vocabulary = []
sparse_vectorspace = {}
doc_word_count = {}
doc_length = {}
total_doc_length = 0
total_term_freq = {}
empty_doc = []
for _id in ids:
    x = x + 1
    response = es.termvectors(index = 'apdata', 
                   doc_type = 'doc', 
                   id = _id, 
                   fields = ["TEXT"], 
                   payloads=False, 
                   positions=False,
                   offsets=False,
                   term_statistics=True,
                   field_statistics=True)
    
    sparse_doc = {}
    word_count = 0
    if(response['term_vectors'] != {}):
        terms = response['term_vectors']['TEXT']['terms']
        for term in terms.keys():
            if term not in vocab.keys():
                vocab[term] = vocab_index
                vocab_index = vocab_index + 1
                reverse_vocabulary.append(term)
            doc_word_count[vocab[term]] = terms[term]['doc_freq']
            total_term_freq[vocab[term]] = terms[term]['ttf']
            sparse_doc[vocab[term]] = terms[term]['term_freq']
            word_count = word_count + terms[term]['term_freq']
    else:
        empty_doc.append(_id)
    sparse_vectorspace[_id] = sparse_doc
    doc_length[_id] = word_count
    total_doc_length = total_doc_length + word_count

    end = time.time()
print(end - start)


301.0149257183075


In [8]:
# for id in doc_word_count:
#     if(doc_word_count[id] == 1):
#         print(reverse_vocabulary[id])

In [9]:
# count = 0
# for vector in sparse_vectorspace:
#     if 615 in vector.keys():
#         count = count + vector[615]
# print(count)

In [10]:
# response = es.termvectors(index = 'apdata', 
#                doc_type = 'doc', 
#                id = 46237, 
#                fields = ["TEXT"], 
#                payloads=False, 
#                positions=False,
#                offsets=False,
#                term_statistics=True,
#                field_statistics=True)

In [11]:
# response['term_vectors']["TEXT"]['terms'].keys()

In [12]:
len(doc_length),len(sparse_vectorspace)

(84679, 84679)

In [13]:
total_doc_length

20984287

In [14]:
okaptf = {}
for vector in sparse_vectorspace.keys():
    terms = sparse_vectorspace[vector]
    doc = {}
    for term in terms.keys():
        doc[term] = (float(terms[term]) / float(terms[term] + 0.5 + (1.5 * float(doc_length[vector] / (total_doc_length / len(doc_length))   ))))
    okaptf[vector] = doc

In [15]:
queriesFile = '/home/parth/information_retrieval/Data/AP89_DATA/AP_DATA/query_desc.51-100.short.txt' 
f = open(queriesFile,"r", encoding='iso-8859-1')
Querytext = f.read()

queries = Querytext.split('\n')[0:-1]
query_dict = {}

for query in queries:
    temp = query.split("   ")
    query_dict[temp[0].split(".")[0]] = temp[1] 
analyse_body = {
  "analyzer": "my_english",
  "text": ""
}

for queryId in query_dict:
    analyse_body["text"] = query_dict[queryId]
    tokens = es.indices.analyze(index = "apdata", body = analyse_body)['tokens']
    query_dict[queryId] = [a['token'] for a in tokens if a['token'] != 'document']
    

In [16]:
okaptf_doc_scores = {}
for doc in query_dict:
    scores = {}
#     print(query_dict[doc])

    for vector in okaptf.keys():
        score = 0
        for word in query_dict[doc]:
            key_dict = okaptf[vector].keys()
            if(vocab[word] in key_dict):
                score = score + okaptf[vector][vocab[word]]
        scores[vector] = score
    okaptf_doc_scores[doc] = scores
    

In [17]:
for query in okaptf_doc_scores:
    okaptf_doc_scores[query] = sorted(okaptf_doc_scores[query].items(), key=lambda kv: kv[1])[::-1][0:1000]

In [18]:
file  = open("okaptf_scores", "w")
for query in okaptf_doc_scores:
    r = 1
    for document in okaptf_doc_scores[query]:
        if(document[1] != 0):
            file.write(str(query) + " Q0 " + doc_no[document[0]] + " " + str(r) + " " + str(document[1]) + " Exp\n" )
        else:
            print(query)
        r = r + 1

In [19]:
tfidf_doc_scores = {}
for doc in query_dict:
    scores = {}
#     print(query_dict[doc])

    for vector in okaptf.keys():
        score = 0
        for word in query_dict[doc]:
            key_dict = okaptf[vector].keys()
            if(vocab[word] in key_dict):
                score = score + (okaptf[vector][vocab[word]] * (log(len(okaptf) / doc_word_count[vocab[word]]) ))
        scores[vector] = score
    tfidf_doc_scores[doc] = scores

In [20]:
for query in tfidf_doc_scores:
    tfidf_doc_scores[query] = sorted(tfidf_doc_scores[query].items(), key=lambda kv: kv[1])[::-1][0:1000]

In [21]:
file  = open("tfidf_scores", "w")
for query in tfidf_doc_scores:
    r = 1
    for document in tfidf_doc_scores[query]:
        if(document[1] != 0):
            file.write(str(query) + " Q0 " + doc_no[document[0]] + " " + str(r) + " " + str(document[1]) + " Exp\n" )
        else:
            print(query)
        r = r + 1

In [22]:
okapi_bm_doc_scores = {}

In [23]:
total_doc = len(ids)
k1 = 1.2
k2 = 1000
b = 0.75
avg_doc_len = total_doc_length / total_doc

In [24]:
avg_doc_len

247.80981116923914

In [25]:

okapi_bm_doc_scores = {}
for doc in query_dict:
    scores = {}
    w_q_count = collections.Counter(query_dict[doc])
    for vector in okaptf.keys():
        score = 0
        key_dict = okaptf[vector].keys()
        terms = sparse_vectorspace[vector]

        for word in query_dict[doc]:
            if(vocab[word] in key_dict):
                w_id = vocab[word]
                log_term = log((total_doc + 0.5) / (doc_word_count[w_id] + 0.5))
                
                word_doc_term_nume = terms[w_id] * (k1 + 1 )
                word_doc_term_den = terms[w_id] + k1 * ((1-b) + b * (doc_length[vector] / avg_doc_len))
                
                word_query_term = w_q_count[word] * ( 1 + k2) / (w_q_count[word] + k2)
                score = score + (log_term * (word_doc_term_nume / word_doc_term_den) * word_query_term)
        scores[vector] = score
    okapi_bm_doc_scores[doc] = scores

In [26]:
for query in okapi_bm_doc_scores:
    okapi_bm_doc_scores[query] = sorted(okapi_bm_doc_scores[query].items(), key=lambda kv: kv[1])[::-1][0:1000]

In [27]:
file  = open("okapi_bm_scores", "w")
for query in okapi_bm_doc_scores:
    r = 1
    for document in okapi_bm_doc_scores[query]:
        if(document[1] != 0):
            file.write(str(query) + " Q0 " + doc_no[document[0]] + " " + str(r) + " " + str(document[1]) + " Exp\n" )
        else:
            print(query)
        r = r + 1

In [28]:
unig_lap = {}
for doc in query_dict:
    scores = {}
    for vector in sparse_vectorspace.keys():
        score = 0
        terms = sparse_vectorspace[vector]
        for word in query_dict[doc]:
            key_dict = terms.keys()
            w_id = vocab[word]
            if(vocab[word] in key_dict):
                score = score + log((terms[w_id] + 1) / (doc_length[vector] + len(vocab)))
            else:
                score = score + log( 1 / (doc_length[vector] + len(vocab)))
        scores[vector] = score
    unig_lap[doc] = scores

In [29]:
 sorted(unig_lap['100'].items(), key=lambda kv: kv[1])[::-1][0:1000]

[('72990', -157.4646807911626),
 ('82078', -157.93849665136415),
 ('20855', -158.18558522716066),
 ('32763', -158.27348165184293),
 ('26348', -158.31256671390136),
 ('8188', -158.97454412351334),
 ('54304', -158.99095456676105),
 ('9325', -159.15331892237933),
 ('67684', -159.22696370485284),
 ('76405', -159.26291002842734),
 ('79723', -159.38103544806904),
 ('68871', -159.40396521470126),
 ('64488', -159.40396521470126),
 ('79938', -159.43055807568328),
 ('46518', -159.43055807568328),
 ('78600', -159.476665204407),
 ('38436', -159.50244840868692),
 ('43133', -159.53123189466217),
 ('13973', -159.61112631787435),
 ('925', -159.63105264334035),
 ('78416', -159.66418125647402),
 ('3237', -159.7353366294395),
 ('36454', -159.76243692105484),
 ('32123', -159.7794687451774),
 ('13739', -159.78247419873412),
 ('77361', -159.79044684162125),
 ('30704', -159.90618096250122),
 ('19396', -159.9068217684401),
 ('3440', -159.91131436038862),
 ('32020', -159.93112225776477),
 ('83577', -159.972335

In [30]:
for query in unig_lap:
    unig_lap[query] = sorted(unig_lap[query].items(), key=lambda kv: kv[1])[::-1][0:1000]

In [31]:
file  = open("unig_lap_scores", "w")
for query in unig_lap:
    r = 1
    for document in unig_lap[query]:
        if(document[1] != 0):
            file.write(str(query) + " Q0 " + doc_no[document[0]] + " " + str(r) + " " + str(document[1]) + " Exp\n" )
        else:
            print(query)
        r = r + 1

In [32]:
l = 0.75
unig_JM = {}
for doc in query_dict:
    scores = {}
    for vector in sparse_vectorspace.keys():
        score = 0
        terms = sparse_vectorspace[vector]
        for word in query_dict[doc]:
            key_dict = terms.keys()
            w_id = vocab[word]
            if(vocab[word] in key_dict):
                p_a  = l * (terms[w_id] /  doc_length[vector])
                p_b = (1 - l) * (total_term_freq[w_id] - terms[w_id]) / ( total_doc_length - doc_length[vector])
                score = score + log(p_a + p_b)
            else:
                p_b = (1 - l) * (total_term_freq[w_id]) / ( total_doc_length - doc_length[vector])
                score = score + log(p_b)
        scores[vector] = score
    unig_JM[doc] = scores



In [33]:
for query in unig_JM:
    unig_JM[query] = sorted(unig_JM[query].items(), key=lambda kv: kv[1])[::-1][0:1000]

In [34]:
file  = open("unig_JM_scores", "w")
for query in unig_JM:
    r = 1
    for document in unig_JM[query]:
        if(document[1] != 0):
            file.write(str(query) + " Q0 " + doc_no[document[0]] + " " + str(r) + " " + str(document[1]) + " Exp\n" )
        else:
            print(query)
        r = r + 1